# Sparse Matrix Recommender (SMR) over larger data

Anton Antonov  
[PythonForPrediction at WordPress](https://pythonforprediction.wordpress.com)   
October 2025

---

## Setup

In [11]:
import pandas
import random
from SparseMatrixRecommender import *
from SparseMatrixRecommender.DataLoaders import *

----

## Get data

In [12]:
dfData0 = load_mushroom_data_frame()

print(dfData0.shape)
dfData0

(8124, 24)


,id,cap-Shape,cap-Surface,cap-Color,bruises?,odor,gill-Attachment,gill-Spacing,gill-Size,gill-Color,...,stalk-Color-Above-Ring,stalk-Color-Below-Ring,veil-Type,veil-Color,ring-Number,ring-Type,spore-Print-Color,population,habitat,edibility
0,id.1,convex,smooth,brown,True,pungent,free,close,narrow,black,...,white,white,partial,white,one,pendant,black,scattered,urban,poisonous
1,id.2,convex,smooth,yellow,True,almond,free,close,broad,black,...,white,white,partial,white,one,pendant,brown,numerous,grasses,edible
2,id.3,bell,smooth,white,True,anise,free,close,broad,brown,...,white,white,partial,white,one,pendant,brown,numerous,meadows,edible
3,id.4,convex,scaly,white,True,pungent,free,close,narrow,brown,...,white,white,partial,white,one,pendant,black,scattered,urban,poisonous
4,id.5,convex,smooth,gray,False,none,free,crowded,broad,black,...,white,white,partial,white,one,evanescent,brown,abundant,grasses,edible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,id.8120,knobbed,smooth,brown,False,none,attached,close,broad,yellow,...,orange,orange,partial,orange,one,pendant,buff,clustered,leaves,edible
8120,id.8121,convex,smooth,brown,False,none,attached,close,broad,yellow,...,orange,orange,partial,brown,one,pendant,buff,several,leaves,edible
8121,id.8122,flat,smooth,brown,False,none,attached,close,broad,brown,...,orange,orange,partial,orange,one,pendant,buff,clustered,leaves,edible
8122,id.8123,knobbed,scaly,brown,False,fishy,free,close,narrow,buff,...,white,white,partial,white,one,evanescent,white,several,leaves,poisonous


----

## Make recommender

In [4]:
smrObj = (SparseMatrixRecommender()
          .create_from_wide_form(dfData0,
                                 item_column_name="id",
                                 columns=["cap-Shape", "cap-Surface", "cap-Color", "bruises?", "odor", "gill-Attachment", "gill-Spacing", "gill-Size", "gill-Color", "edibility"],
                                 add_tag_types_to_column_names=True,
                                 tag_value_separator=":")
          .apply_term_weight_functions("IDF", "None", "Cosine"))

The obtained recommender object:

In [6]:
smrObj

<Sparse matrix recommender object with matrix dimensions 8124x51
	and with 10 tag types>

Recommender matrix sample:

In [8]:
sample_rows = random.sample(smrObj.take_M().row_names(), 10)
smrObj.take_M()[sample_rows, :5].print_matrix()

        |    cap-Shape:bell cap-Shape:conical  cap-Shape:convex    cap-Shape:flat cap-Shape:knobbed
---------------------------------------------------------------------------------------------------
id.1210 |                 .                 .               1.0                 .                 .
 id.218 |                 .                 .               1.0                 .                 .
id.4969 |                 .                 .                 .               1.0                 .
id.8101 |                 .                 .                 .               1.0                 .
id.6679 |                 .                 .               1.0                 .                 .
id.2512 |                 .                 .               1.0                 .                 .
id.4606 |                 .                 .               1.0                 .                 .
id.4496 |                 .                 .                 .               1.0                 .


Recommend by profile:

In [9]:
recs = (smrObj
        .recommend_by_profile({"cap-Shape:convex": 1.2, "edibility:poisonous": 1.4}, nrecs=12)
        .join_across(dfData0, on="id")
        .take_value())

recs

,id,Score,cap-Shape,cap-Surface,cap-Color,bruises?,odor,gill-Attachment,gill-Spacing,gill-Size,...,stalk-Color-Above-Ring,stalk-Color-Below-Ring,veil-Type,veil-Color,ring-Number,ring-Type,spore-Print-Color,population,habitat,edibility
0,id.1,1.0,convex,smooth,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,scattered,urban,poisonous
1,id.1004,1.0,convex,smooth,white,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,several,grasses,poisonous
2,id.1032,1.0,convex,scaly,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,several,grasses,poisonous
3,id.1040,1.0,convex,smooth,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,scattered,urban,poisonous
4,id.1100,1.0,convex,scaly,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,brown,scattered,grasses,poisonous
5,id.1104,1.0,convex,scaly,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,brown,scattered,grasses,poisonous
6,id.1107,1.0,convex,scaly,white,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,several,urban,poisonous
7,id.1133,1.0,convex,smooth,brown,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,black,scattered,grasses,poisonous
8,id.115,1.0,convex,scaly,white,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,brown,several,urban,poisonous
9,id.1153,1.0,convex,scaly,white,True,pungent,free,close,narrow,...,white,white,partial,white,one,pendant,brown,several,grasses,poisonous


Recommend by history:

In [10]:
recs = (smrObj
        .recommend("id.10", nrecs=12, remove_history=False)
        .join_across(dfData0, on="id")
        .take_value())

recs

,id,Score,cap-Shape,cap-Surface,cap-Color,bruises?,odor,gill-Attachment,gill-Spacing,gill-Size,...,stalk-Color-Above-Ring,stalk-Color-Below-Ring,veil-Type,veil-Color,ring-Number,ring-Type,spore-Print-Color,population,habitat,edibility
0,id.10,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,black,scattered,meadows,edible
1,id.160,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,black,scattered,grasses,edible
2,id.296,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,brown,numerous,meadows,edible
3,id.315,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,brown,numerous,grasses,edible
4,id.561,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,brown,scattered,grasses,edible
5,id.658,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,black,numerous,grasses,edible
6,id.811,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,black,numerous,meadows,edible
7,id.894,1.0,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,brown,scattered,meadows,edible
8,id.1022,0.9,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,black,numerous,meadows,edible
9,id.1128,0.9,bell,smooth,yellow,True,almond,free,close,broad,...,white,white,partial,white,one,pendant,brown,numerous,grasses,edible
